## Основная информация по базе знаний:

*Профессия нейро-сотрудника* - консультант по техническим характеристикам компьютера, который по вашим потребностям сможет подобрать подходящие комплектующие для компьютера.

В качестве базы знаний я буду использовать **Google-документ**, структура которого будет выглядеть следующим образом:
* Заголовок 1 уровня (логическое описание, тема к которой относиться фрагмент);
* Заголовок 2 уровня (отражает смысл фрагмента или группы, в которую входит фрагмент);
* Фрагмент (основной и более подробный текст).

Также в инструкции для нейро-сотрудника (**Google-документ**) и промпте я решил описать желаемую структуру, которая должна быть на входе языковой модели, входными данными которой является **Google-документ**.

Для реализации нейро-сотрудника я буду использовать фреймворк **LlamaIndex**.

## Решение задачи:

Скачаем в папку *content* инструкцию для нейро-сотрудника:

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/document/d/19rk3rnk1ReQggoyO3F_1kr_1ClWCRf3OIzy3gChgOPA/export?format=txt' -O /content/info.txt

--2025-08-20 19:28:09--  https://docs.google.com/document/d/19rk3rnk1ReQggoyO3F_1kr_1ClWCRf3OIzy3gChgOPA/export?format=txt
Resolving docs.google.com (docs.google.com)... 74.125.199.113, 74.125.199.100, 74.125.199.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.199.113|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0s-30-docstext.googleusercontent.com/export/e6hpso97lrhpva19l3uocm525o/2uicb469j4j7ob8f0c75seksj0/1755718090000/102527026341015577601/*/19rk3rnk1ReQggoyO3F_1kr_1ClWCRf3OIzy3gChgOPA?format=txt [following]
--2025-08-20 19:28:10--  https://doc-0s-30-docstext.googleusercontent.com/export/e6hpso97lrhpva19l3uocm525o/2uicb469j4j7ob8f0c75seksj0/1755718090000/102527026341015577601/*/19rk3rnk1ReQggoyO3F_1kr_1ClWCRf3OIzy3gChgOPA?format=txt
Resolving doc-0s-30-docstext.googleusercontent.com (doc-0s-30-docstext.googleusercontent.com)... 74.125.135.132, 2607:f8b0:400e:c01::84
Connecting to doc-0s-30-docstext.googleuse

Установим необходимые библиотеки:

In [ ]:
!pip install llama-index-core "arize-phoenix[evals,llama-index]" gcsfs nest-asyncio "openinference-instrumentation-llama-index>=2.0.0"
!pip install llama_index --upgrade
!pip install transformers sentence-transformers accelerate bitsandbytes llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install nemoguardrails
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

  Using cached llama_index-0.11.0-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_cli-0.3.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.11.0.post1-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_embeddings_openai-0.2.5-py3-none-any.whl.metadata (686 bytes)
  Using cached llama_index_llms_openai-0.2.16-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_readers_file-0.2.2-py3-none-any.whl.metadata (5.4 kB)
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached llama_index_llms_openai-0.2.15-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_llms_openai-0.2.14-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_llms_openai-0.2.13-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_llms_openai-0.2.12-py3-none-any.whl.metadata (649 bytes)
  Using cached llama_index_llms_openai-0.2

Импортируем необходимые библиотеки и укажем токен из **Hugging Face** для дальнейшего его использования в коде. Код можно сгенерировать в настройках профиля:

In [ ]:
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
import nest_asyncio
import threading
import asyncio
import torch
import re
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
    Document,
    StorageContext
)
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.ingestion import IngestionPipeline
import phoenix as px
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from huggingface_hub import login
from llama_index.core.postprocessor import LongContextReorder
from nemoguardrails import RailsConfig, LLMRails

HF_TOKEN = "your_token"
login(token=HF_TOKEN)

Создадим функцию `clean_document_text`, которая очищает текст от проблемных символов в числах:

In [ ]:
def clean_document_text(text):
    if not isinstance(text, str):
        return text
    #заменяем неразрывные пробелы на обычные
    text = text.replace('\u202f', ' ')  # Thin space
    text = text.replace('\xa0', ' ')    # Non-breaking space
    #удаляем лишние пробелы между цифрами
    text = re.sub(r'(\d)\s+(\d)', r'\1\2', text)
    return text

Выполним загрузку нашей базы знаний в папку content, применим функцию `clean_document_text` и упростим метаданные:

In [ ]:
raw_documents = SimpleDirectoryReader('/content').load_data() #загружаем документы и очищаем

#создаем новый список очищенных документов
cleaned_documents = []
for doc in raw_documents:
    cleaned_text = clean_document_text(doc.text)
    cleaned_doc = Document(
        text=cleaned_text,
        metadata=doc.metadata.copy() if hasattr(doc, 'metadata') and doc.metadata else {}
    )
    cleaned_documents.append(cleaned_doc)

#упрощаем метаданные
for doc in cleaned_documents:
    doc.metadata = {
        "source": doc.metadata.get("source", "unknown"),
        "page_label": doc.metadata.get("page_label", "0")
    }

documents = cleaned_documents

**Phoenix** использует **asyncio**, поэтому нам необходимо выполнить слепдующую команду, чтобы избежать ошибку:

In [ ]:
nest_asyncio.apply() #необходим для параллельных вычислений в среде ноутбуков

Запустим **Phoenix** в фоновом режиме для сбора данных трассировки:

In [ ]:
session = px.launch_app()

/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit https://6iliv7f44ul1-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


Напишем функцию, которая производит запуск **Cloudflared** туннеля.

Внутри функции с помощью subprocess.Popen запускается команда .`/cloudflared tunnel --url http://localhost:6006`. Эта команда создаёт защищённый публичный туннель к локальному веб-серверу **Phoenix UI**, который работает на порту 6006.

Когда появляется строка с публичным адресом (он содержит "trycloudflare.com" — например, https://randomstring.trycloudflare.com), с помощью регулярного выражения извлекается URL. Этот URL выводится в консоль — по нему можно открыть Phoenix UI из любого браузера.

In [ ]:
#запуск Cloudflared туннеля
def start_cloudflared():
    import subprocess, re
    proc = subprocess.Popen(
        ['./cloudflared', 'tunnel', '--url', 'http://localhost:6006'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )
    for line in proc.stdout:
        if "trycloudflare.com" in line:
            urls = re.findall(r'https?://[^\s]+', line)
            if urls:
                print("\n🌐 Phoenix UI доступен по публичной ссылке:", urls[0])
                break


threading.Thread(target=start_cloudflared, daemon=True).start() #запускаем функцию в отдельном фоне потока, чтобы основной код не блокировался

Настройка трассировки (Tracing) для **Phoenix** и **LlamaIndex**:

In [ ]:
#настройка трассировки с обходом проблемы сериализации
endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

#инициализация инструментации с обработкой ошибок
try:
    LlamaIndexInstrumentor().instrument(
        skip_dep_check=True,
        tracer_provider=tracer_provider
    )
except Exception as e:
    print(f"⚠️ Предупреждение: ошибка инициализации инструментации: {str(e)}")

Этот код настраивает квантование модели для загрузки в *4-битной точности* с использованием библиотеки **BitsAndBytes**. Включены двойное квантование и вычисления в формате *float16* для повышения эффективности и снижения потребления памяти:

In [ ]:
#конфигурация модели
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

Функция `completion_to_prompt` определяет системный промпт и шаблон, который указан на странице модели **IlyaGusev/saiga_yandexgpt_8b**:

In [ ]:
def completion_to_prompt(completion: str) -> str:
    system_prompt = (
        "Ты — технический эксперт. Отвечай ТОЛЬКО на основе предоставленного документа."
        "Если информации нет в документе, скажи 'В документе нет информации по этому вопросу'. "
        "Не добавляй свои знания или предположения."
        "Будь максимально точен и избегай технических неточностей."
    )
    return (
        f"<|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|>"
        f"<|start_header_id|>user<|end_header_id|>\n\n{completion}<|eot_id|>"
        f"<|start_header_id|>assistant<|end_header_id|>\n\n"
    )

Загрузим токенизатор и определим параметры конфига:

In [ ]:
#инициализация русскоязычной модели
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_yandexgpt_8b", token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    "IlyaGusev/saiga_yandexgpt_8b",
    device_map="auto",
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
    token=HF_TOKEN
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Убедимся, что токенизатор знает специальные токены:

In [ ]:
special_tokens = {
    "additional_special_tokens": [
        "<|start_header_id|>",
        "<|end_header_id|>",
        "<|eot_id|>"
    ]
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(129024, 4096)

Следующий код создаёт и настраивает объект языковой модели (**LLM**) из библиотеки **Hugging Face** для генерации текста, задавая конкретные параметры:

In [ ]:
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    context_window=2048,
    max_new_tokens=512,
    generate_kwargs={
        "do_sample": False,
        "repetition_penalty": 1.3,
        "no_repeat_ngram_size": 4,
        "num_beams": 1,
        "temperature": None,
        "top_p": None
    },
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)

Этот код настраивает систему автоматической модерации запросов, которая с помощью заданных правил (**rails**) и языковой модели (**LLM**) блокирует вопросы, связанные с упоминанием цены или стоимости.

Код анализирует входящий вопрос пользователя и возвращает **False** (небезопасно), если обнаруживает запрещённую тему, в противном случае — **True**:

In [ ]:
#создаем простую конфигурацию rails
config = RailsConfig.from_content("""
define user ask price
  "price"
  "cost"
  "how much"
  "за сколько"
  "цена"
  "ценовой диапазон"
  "сколько стоит"
  "buy for"  # Примеры на английском и русском для запросов о цене

define flow
  user ask price
  bot inform price content

define bot inform price content
  "Обнаружен запрос о цене или ценовом диапазоне. Запрос заблокирован."
""")

rails = LLMRails(config, llm=llm) #инициализируем rails с LLM

#проверяем запрос на безопасность с помощью NeMo Guardrails
async def moderate_with_nemo_guard(question: str) -> bool:
    response = await rails.generate_async(messages=[{"role": "user", "content": question}])
    #если rails вернул сообщение о цене — unsafe
    if "Обнаружен запрос о цене" in response["content"]:
        return False
    return True

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Выполним инициализацию эмбеддинг-модели:

In [ ]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cuda",
    max_length=256,
    token=HF_TOKEN,
    embed_batch_size=128
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Этот код создаёт и настраивает инструмент для разделения текста на части (чанки) по токенам, а не по символам или словам.

Он разбивает текст на фрагменты размером до 1024 токенов с перекрытием в 64 токена для сохранения контекста, используя в качестве разделителей сначала переносы строк, а затем пробелы и табуляцию:

In [ ]:
text_splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    separator="\n",
    backup_separators=[" ", "\t"]
)

Устанавливаем глобальные настройки:

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

Создаётся конвейер обработки документов, который разбивает исходные документы на фрагменты (узлы) с помощью текстового сплиттера, а затем создаёт для каждого фрагмента векторные представления (эмбеддинги) с использованием embedding-модели.

Полученные узлы с текстом и соответствующими эмбеддингами готовы для последующего индексирования:

In [ ]:
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        embed_model
    ]
)

nodes = pipeline.run(documents=documents, show_progress=True)

print(f"Создано {len(nodes)} узлов для индексирования")

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/31355 [00:00<?, ?it/s]

Создано 31355 узлов для индексирования


Создание `VectorStoreIndex` из **nodes** (уже с эмбеддингами):

In [ ]:
index = VectorStoreIndex(nodes, show_progress=True)

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

...

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Этот код создаёт объект для специального переупорядочивания частей длинного текста, чтобы улучшить работу языковых моделей с большими контекстами.

Алгоритм стратегически перемешивает фрагменты текста, чтобы наиболее релевантная информация находилась в начале и конце контекста, что повышает точность обработки моделью:

In [ ]:
reorder = LongContextReorder()

Выполним небольшую оптимизацию памяти:

In [ ]:
del documents  # Удаляем исходные документы
del cleaned_documents  # Удаляем очищенные документы
del raw_documents  # Удаляем временные необработанные документы
del nodes  # Удаляем узлы, так как они уже в индексе
gc.collect()  # Первый вызов сборщика мусора
torch.cuda.synchronize()  # Синхронизация GPU для завершения операций
torch.cuda.empty_cache()  # Очистка кэша GPU
gc.collect()  # Финальный сбор мусора

0

Этот код создаёт инструмент для выполнения запросов к индексированным данным:

In [ ]:
query_engine = index.as_query_engine(
    streaming=False,
    similarity_top_k=5,
    node_postprocessors=[reorder]
)

Проверяет запрос на наличие ключевых слов, связанных с ценой:

In [ ]:
def is_unsafe(question: str) -> bool:
    price_keywords = [
        "price", "cost", "how much", "buy for",
        "за сколько", "цена", "ценовой диапазон", "сколько стоит"
    ]
    lower_question = question.lower()
    for keyword in price_keywords:
        if keyword in lower_question:
            return True
    return False

Обертка для безопасного выполнения запросов с фильтрацией цен:

In [ ]:
def safe_query(question):
    if is_unsafe(question):
        return "Обнаружен запрос о цене или ценовом диапазоне. Запрос заблокирован."
    try:
        response = query_engine.query(question)
        return str(response) #преобразуем в строку для избежания проблем сериализации
    except Exception as e:
        return f"Ошибка при обработке запроса: {str(e)}"

Напишем 2 промпта для уже готовой модели:

In [ ]:
response_1 = safe_query("Вы хорошо разбираетесь в технических характеристиках и комплектующих компьютера. За сколько рублей можно купить игровой компьютер? Ответ должен быть основан только на информации из инструкции, ничего не выдумывайте. Ответ должен быть ясным и кратким. Ценовой диапазон должен быть представлен в рублях и в числовом формате.")
print("\n🔍 Ответ 1:", response_1)

response_2 = safe_query("Вы хорошо разбираетесь в технических характеристиках и комплектующих компьютера. Расскажи мне про влияние объёма и скорости RAM на производительность. Ответ должен быть основан только на информации из инструкции, ничего не выдумывайте. Ответ должен быть ясным и кратким.")
print("\n🔍 Ответ 2:", response_2)


🔍 Ответ 1: Обнаружен запрос о цене или ценовом диапазоне. Запрос заблокирован.

🔍 Ответ 2: Объём и скорость оперативной памяти существенно влияют на общую производительность компьютерных устройств, особенно когда речь идёт об играх. Рассмотрим эти факторы подробнее исходя исключительно из представленной документации.

**Влияние объема ОЗУ:**  
Объем оперативной памяти критически важен для многозадачности – одновременного выполнения нескольких приложений без замедления реакции устройства. В контексте игры это означает быструю загрузку уровнями внутри самой программы, эффективное управление данными ОС и возможность параллельной эксплуатации других ресурсов. Минимальный рекомендуемый объем составляет **минимум 8 Гб**, но большинство экспертов советуют иметь как можно больше доступной оперативы - от стандартных наборов размером около **16 Gb** до более продвинутых конфигураций объёмом свыше **30–32 Gb**. Это позволяет обеспечить плавную работу даже самых требовательных современных проекто

# Вывод:






На *1 промпт* модель отреагировала корректно, заблокировав запрос, так как была установлена соответствующая фильтрация на определенную тему, а именно: все, что связано с ценами.

На *2 промпт* модель ответила развернуто и правильно, опираясь на базу знаний из *Google-документа*.

У нас получилось создать **RAG-систему**, в основе которой **LLM - IlyaGusev/saiga_yandexgpt_8b**. В ней реализована возможность просмотра трассировки через **Phonex UI**, фильтрация данных и тщательная обработка входных данных. Также для более корректной и лучшей работы *RAG-системы* были реализованы следующие подходы, закрывающие проблему **"болевых точек"**:
*   **Болевая точка 1:** недостающий контент (реализовал очистку данных и улучшение промптов);
*   **Болевая точка 4:** контекст не извлечен (использовался **ресортировщик контента**);
*   **Болевая точка 8:** масштабируемость полученных данных;
*   **Болевая точка 12:** Безопасность **LLM** (через **NeMo Guardrails**).